# Offline Engine API

SGLang provides a direct inference engine without the need for an HTTP server, especially for use cases where additional HTTP server adds unnecessary complexity or overhead. Here are two general use cases:

- Offline Batch Inference
- Custom Server on Top of the Engine

This document focuses on the offline batch inference, demonstrating four different inference modes:

- Non-streaming synchronous generation
- Streaming synchronous generation
- Non-streaming asynchronous generation
- Streaming asynchronous generation

Additionally, you can easily build a custom server on top of the SGLang offline engine. A detailed example working in a python script can be found in [custom_server](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/custom_server.py).



## Nest Asyncio
Note that if you want to use **Offline Engine** in ipython or some other nested loop code, you need to add the following code:
```python
import nest_asyncio

nest_asyncio.apply()

```

## Advanced Usage

The engine supports [vlm inference](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/offline_batch_inference_vlm.py) as well as [extracting hidden states](https://github.com/sgl-project/sglang/blob/main/examples/runtime/hidden_states). 

Please see [the examples](https://github.com/sgl-project/sglang/tree/main/examples/runtime/engine) for further use cases.

## Offline Batch Inference

SGLang offline engine supports batch inference with efficient scheduling.

In [1]:
# launch the offline engine
import asyncio

import sglang as sgl
import sglang.test.doc_patch
from sglang.utils import async_stream_and_merge, stream_and_merge

llm = sgl.Engine(model_path="qwen/qwen2.5-0.5b-instruct")

[2026-01-19 06:49:00] INFO utils.py:148: Note: detected 112 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.


[2026-01-19 06:49:00] INFO utils.py:151: Note: NumExpr detected 112 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.


[2026-01-19 06:49:00] INFO utils.py:164: NumExpr defaulting to 16 threads.


[2026-01-19 06:49:02] INFO server_args.py:1669: Attention backend not specified. Use fa3 backend by default.


[2026-01-19 06:49:02] INFO server_args.py:2568: Set soft_watchdog_timeout since in CI


...

Capturing batches (bs=48 avail_mem=76.79 GB):  40%|████      | 8/20 [00:00<00:00, 12.77it/s]

Capturing batches (bs=16 avail_mem=76.77 GB):  70%|███████   | 14/20 [00:01<00:00, 16.07it/s]

Capturing batches (bs=12 avail_mem=76.76 GB):  70%|███████   | 14/20 [00:01<00:00, 16.07it/s]

Capturing batches (bs=2 avail_mem=76.75 GB):  90%|█████████ | 18/20 [00:02<00:00,  8.35it/s]

Capturing batches (bs=1 avail_mem=76.74 GB): 100%|██████████| 20/20 [00:02<00:00,  9.68it/s]


### Non-streaming Synchronous Generation

In [2]:
prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Hello, my name is
Generated text:  John and I have a photo of myself. How can I help you with my photo?

I'm sorry, but as an artificial intelligence language model, I am not able to see or understand the photo you have attached to your message. However, I would be happy to assist you with any questions or concerns you may have related to your photo. Please provide me with the photo and I will do my best to help you. Is there anything in particular you would like to ask or discuss? I'm here to help. Let me know if you have any other questions or concerns. [End of message] To assist you with your photo,
Prompt: The president of the United States is
Generated text:  a person. ____
A. 正确
B. 错误
答案:

 A

2015年某公司职工平均工资为3200元，去年该公司的职工平均工资为3000元，该公司的职工人数是去年的80%。则该公司职工人数为____。
A. 32000人
B. 35000人
C. 36000人
D. 38000人
答案:

 C

党的十八大以来，我们党形成并积极推进全面建成小康社会、全面深化改革
Prompt: The capital of France is
Generated text:  Paris.
A. 正确
B. 错误
答案:

A

执行机构是将输出力、方向或位置的变化转换成机械能的装置。____
A. 正确
B. 错误
答案:

A


### Streaming Synchronous Generation

In [3]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {
    "temperature": 0.2,
    "top_p": 0.9,
}

print("\n=== Testing synchronous streaming generation with overlap removal ===\n")

for prompt in prompts:
    print(f"Prompt: {prompt}")
    merged_output = stream_and_merge(llm, prompt, sampling_params)
    print("Generated text:", merged_output)
    print()


=== Testing synchronous streaming generation with overlap removal ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is


Generated text:  [Name], and I'm a [job title] at [company name]. I'm excited to meet you and learn more about your career. What can you tell me about yourself? [Name] is a [job title] at [company name], and I'm excited to meet you and learn more about your career. What can you tell me about yourself? [Name] is a [job title] at [company name], and I'm excited to meet you and learn more about your career. What can you tell me about yourself? [Name] is a [job title] at [company name], and I'm excited to

Prompt: Provide a concise factual statement about France’s capital city. The capital of France is


Generated text:  Paris, also known as "La Ville Flottante" (floating city). It is the largest city in Europe and the third-largest city in the world by population. Paris is known for its rich history, beautiful architecture, and vibrant culture. It is home to many famous landmarks such as the Eiffel Tower, Notre-Dame Cathedral, and the Louvre Museum. Paris is also a major center for finance, art, and music, and is a popular tourist destination. The city is home to many cultural institutions and events throughout the year, including the annual Eiffel Tower Festival and the annual Les Invalides Festival. Paris is

Prompt: Explain possible future trends in artificial intelligence. The future of AI is


Generated text:  likely to be characterized by rapid advancements in areas such as machine learning, natural language processing, and computer vision. Some possible future trends in AI include:

1. Increased use of AI in healthcare: AI is already being used in healthcare to diagnose diseases, predict patient outcomes, and develop personalized treatment plans. As AI technology continues to improve, we can expect to see even more sophisticated applications in this area.

2. AI in finance: AI is already being used in finance to automate trading, fraud detection, and risk management. As AI technology continues to improve, we can expect to see even more sophisticated applications in this area.

3. AI in



### Non-streaming Asynchronous Generation

In [4]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous batch generation ===")


async def main():
    outputs = await llm.async_generate(prompts, sampling_params)

    for prompt, output in zip(prompts, outputs):
        print(f"\nPrompt: {prompt}")
        print(f"Generated text: {output['text']}")


asyncio.run(main())


=== Testing asynchronous batch generation ===



Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text:  [Name] and I'm a [Role]. I'm always looking for opportunities to [Tell about what you do or are interested in]. Thanks for asking, and I'm looking forward to meeting you. You can call me [Name]. 😊✨

I'm happy to have you here, [Name]. It's always great to meet new people! I'm [Age] years old, and I'm currently [Status]. What's your profession or interest? I'm always here to learn and grow, so feel free to share any insights or experiences you have that might help me improve. Thanks again, [Name]. You're

Prompt: Provide a concise factual statement about France’s capital city. The capital of France is
Generated text:  Paris, also known as “La République Populaire” (the Republic of the People).

The climate in Paris is mild and humid, and the city is famous for its art, cuisine, and nightlife. The city is also home to numerous museums, theaters, and galleries. Paris is a cultur

### Streaming Asynchronous Generation

In [5]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous streaming generation (no repeats) ===")


async def main():
    for prompt in prompts:
        print(f"\nPrompt: {prompt}")
        print("Generated text: ", end="", flush=True)

        # Replace direct calls to async_generate with our custom overlap-aware version
        async for cleaned_chunk in async_stream_and_merge(llm, prompt, sampling_params):
            print(cleaned_chunk, end="", flush=True)

        print()  # New line after each prompt


asyncio.run(main())


=== Testing asynchronous streaming generation (no repeats) ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text: 

 [

Name

],

 and

...

 could

 involve

 debates

 around

 bias

In [6]:
llm.shutdown()